In [169]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

block_size = 16
batch_size = 32
max_iters = 3000
learning_rate = 3e-4
eval_iters = 25
dropout = 0.2
n_embd = 384
n_layer = 4
n_head = 4

SAVE_PATH = "model_shakespear.pth"

cpu


In [151]:
with open('julius_caesar.txt', 'r', encoding='utf-8') as f:
          text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '’', '“', '”', '\ufeff']
71


In [152]:
string_to_int = { ch: i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)

[47, 44, 51, 51, 54]
hello


In [153]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14, 16, 25, 16,  1, 20,  8,  1, 29,
        54, 52, 44,  8,  1, 12,  1, 58, 59, 57, 44, 44, 59,  8,  0,  0,  0, 16,
        53, 59, 44, 57,  1, 17, 51, 40, 61, 48, 60, 58,  6,  1, 24, 40, 57, 60,
        51, 51, 60, 58,  1, 40, 53, 43,  1, 40,  1, 59, 47, 57, 54, 53, 46,  1,
        54, 45,  1, 14, 48, 59, 48, 65, 44, 53, 58,  8,  0,  0, 17, 23, 12, 33,
        20, 32, 30,  8,  0, 19, 44, 53, 42, 44])


In [154]:
n = int(len(data)*0.8)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[57, 58,  6,  0, 25, 44, 61, 44, 57,  1, 51, 40, 42, 50, 58,  1],
        [14, 20, 25, 25, 12,  8,  0, 15, 48, 57, 44, 42, 59, 51, 64,  6],
        [51,  1, 58, 44, 53, 43,  1, 24, 40, 57, 50,  1, 12, 53, 59, 54],
        [ 1, 52, 54, 54, 53,  6,  0, 31, 47, 40, 53,  1, 58, 60, 42, 47],
        [30, 20, 32, 30,  8,  0, 34, 47, 40, 59,  1, 44, 53, 59, 44, 57],
        [53, 64,  8,  0,  0, 12, 25, 31, 26, 25, 35,  8,  0, 14, 40, 44],
        [40, 64,  1, 62, 48, 59, 47, 54, 60, 59,  1, 59, 47, 44,  1, 58],
        [54, 60, 51, 43,  1, 57, 60, 53,  1, 59, 54,  1, 59, 47, 44, 58],
        [45, 40, 42, 60, 51, 59, 48, 44, 58,  6,  0, 31, 54,  1, 52, 54],
        [47, 44, 64,  1, 41, 44, 47, 54, 51, 43,  1, 40,  1, 46, 57, 44],
        [36, 16, 25,  8,  0, 26,  1, 55, 48, 59, 44, 54, 60, 58,  1, 58],
        [ 1, 62, 48, 59, 47,  1, 47, 48, 52,  6,  1, 13, 57, 60, 59, 60],
        [54, 60, 57,  1, 47, 54, 60, 58, 44, 58,  6,  1, 45, 40, 51, 51],
        [ 1, 47, 48, 52,  1, 

In [155]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, target is {target}")

When input is tensor([70]), target is 12
When input is tensor([70, 12]), target is 14
When input is tensor([70, 12, 14]), target is 31
When input is tensor([70, 12, 14, 31]), target is 1
When input is tensor([70, 12, 14, 31,  1]), target is 20
When input is tensor([70, 12, 14, 31,  1, 20]), target is 0
When input is tensor([70, 12, 14, 31,  1, 20,  0]), target is 0
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0]), target is 30
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30]), target is 14
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14]), target is 16
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14, 16]), target is 25
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14, 16, 25]), target is 16
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14, 16, 25, 16]), target is 1
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14, 16, 25, 16,  1]), target is 20
When input is tensor([70, 12, 14, 31,  1, 20,  0,  0, 30, 14, 16, 

In [156]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [157]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [158]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

In [159]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [160]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [161]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
model = model.to(device)
print(model)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

GPTLanguageModel(
  (token_embedding_table): Embedding(71, 384)
  (position_embedding_table): Embedding(16, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=384, out_features=96, bias=False)
            (query): Linear(in_features=384, out_features=96, bias=False)
            (value): Linear(in_features=384, out_features=96, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=T

In [162]:
# Create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in tqdm(range(max_iters)):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter} | train loss: {losses['train']:.3f} | val loss: {losses['val']:.3f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

  0%|                                                                                                                               | 0/3000 [00:00<?, ?it/s]

step: 0 | train loss: 4.353 | val loss: 4.350


  1%|█                                                                                                                     | 26/3000 [00:08<35:49,  1.38it/s]

step: 25 | train loss: 2.650 | val loss: 2.691


  2%|██                                                                                                                    | 51/3000 [00:13<31:44,  1.55it/s]

...

step: 2950 | train loss: 1.499 | val loss: 1.726


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 2976/3000 [12:11<00:17,  1.39it/s]

step: 2975 | train loss: 1.469 | val loss: 1.684


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [12:15<00:00,  4.08it/s]

1.6249074935913086


In [168]:
torch.save(model.state_dict(), SAVE_PATH)

In [171]:
model_load = GPTLanguageModel(vocab_size)
model_load.load_state_dict(torch.load(SAVE_PATH))

<All keys matched successfully>

In [174]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model_load.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


It goes him Troanina laulate! What I, should, and you shall be grooks for; our had wounder that fuly hone. I madry his of Trie nace’d,
As dreams off live. I. I shar live, be that was these says are to this.

CASSIUS.
PriFe hear whil; all your honourabl, hone toumper.

LLUS.
Truly. Enter not all
A bad in thus narre live his aouthful?

PORTIA.
Let me what magain.
Enterius, whice’s, it it fauls.
Whow more what thear tempanion, andred Pablucius._]

SER.
O, or the can some,
Whigh a sickne.

SERVANT.

